In [ ]:
#Prior to execute this notebook, please make sure you have already cloned the sample data
#!git clone https://github.com/xieyongliang/all-in-one-ai-sample-data.git ../../../all-in-one-ai-sample-data

In [2]:
import sagemaker
from sagemaker.pytorch import PyTorch

In [3]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
!rm -rf sourcedir.tar.gz
!tar czvf sourcedir.tar.gz *

In [ ]:
source_dir = 's3://{0}/{1}/source/'.format(bucket, 'yolov5')
!aws s3 cp sourcedir.tar.gz $source_dir
source_dir = '{0}sourcedir.tar.gz'.format(source_dir)
!rm -f sourcedir.tar.gz

In [ ]:
train_dir = 's3://{0}/{1}/data/train'.format(bucket, 'yolov5')
!aws s3 cp ../../../all-in-one-ai-sample-data/yolov5/train/ $train_dir --recursive

In [16]:
images_dir = 's3://{0}/{1}/data/train/images'.format(bucket, 'yolov5')
labels_dir = 's3://{0}/{1}/data/train/labels'.format(bucket, 'yolov5')
cfg_dir = 's3://{0}/{1}/data/train/cfg'.format(bucket, 'yolov5')

In [17]:
job_name = None
entry_point = 'train.py'
git_config = {'repo': 'https://github.com/ultralytics/yolov5.git', 'branch': 'v6.2'}
role = role
hyperparameters = {
    'data': '/opt/ml/input/data/cfg/data.yaml', 
    'cfg': 'yolov5s.yaml', 
    'weight': 'yolov5s.pt', 
    'project': '/opt/ml/model/',
    'name': 'tutorial', 
    'img': 640, 
    'batch': 16, 
    'epochs': 100,
    'device': 0
}
framework_version = '1.12'
py_version = 'py38'
instance_type = 'ml.g4dn.16xlarge'
instance_count = 1
inputs = {
    'images': images_dir,
    'labels': labels_dir,
    'cfg': cfg_dir
}

In [18]:
estimator = PyTorch(
    entry_point = entry_point,
    source_dir = '.',
    git_config = git_config,
    role = role,
    debugger_hook_config = False,
    hyperparameters = hyperparameters,
    framework_version = framework_version, 
    py_version = py_version,
    instance_type = instance_type,
    instance_count = instance_count
)

In [ ]:
estimator.fit(inputs, job_name = job_name)

In [21]:
training_job_name = estimator.latest_training_job.name

In [22]:
model_name = None
model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name)
entry_point = 'inference.py'
framework_version = '1.10.2'
py_version = 'py38'
model_environment = {}

In [23]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = source_dir,
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

In [24]:
endpoint_name = None
instance_type = 'ml.m5.xlarge'
instance_count = 1

In [ ]:
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count
)

In [17]:
from os import listdir
from os.path import isfile, join

sample_dir ='../../../all-in-one-ai-sample-data/yolov5/train/images/test'
files = [f for f in listdir(sample_dir) if isfile(join(sample_dir, f))]

bytes  = open(join(sample_dir, files[0]), 'rb').read()

In [ ]:
from sagemaker.serializers import DataSerializer 
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = DataSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(bytes, initial_args={"ContentType":"image/png"})

In [19]:
predictor.delete_endpoint()